In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np

# Solar

In [ ]:
lista = os.listdir('/Users/cchallu/Downloads/pa-pv-2006')
df = []
for file in lista[:5]:
    data = pd.read_csv(f'/Users/cchallu/Downloads/pa-pv-2006/{file}')
    unique_id = file.split('.')[0] + '_' + file.split('.')[1]+ '_' + file.split('.')[2]
    data.columns = ['ds', 'y']
    data['unique_id'] = unique_id
    df.append(data)
df = pd.concat(df).reset_index(drop=True)
df = df[['unique_id', 'ds', 'y']]

In [ ]:
df.to_csv('solar_pa.csv', index=False)

# Air quality

In [ ]:
data_air = pd.read_csv('new_data/air_quality.csv') # Download from https://github.com/WenjieDu/TSDB
data_air['ds'] = data_air['year'].astype(str) + '-' + data_air['month'].astype(str).str.zfill(2) + '-' + data_air['day'].astype(str).str.zfill(2) + ' ' + data_air['hour'].astype(str).str.zfill(2) + ':00'
data_air['ds'] = pd.to_datetime(data_air['ds'])

df = []
columns = ['PM2.5', 'PM10', 'TEMP']
for column in columns:
    data_filter = data_air[['station', 'ds', column]]
    data_filter['unique_id'] = data_filter['station'] + '_' + column
    data_filter = data_filter[['unique_id', 'ds', column]]
    data_filter = data_filter.rename(columns={column: 'y'})
    data_filter['y'] = data_filter['y'].ffill()
    df.append(data_filter)
data = pd.concat(df)

In [ ]:
data.to_csv('new_data/airquality.csv', index=False)

In [ ]:
data_plot = data[data['unique_id']==data['unique_id'].unique()[np.random.randint(data['unique_id'].nunique())]]
print('len', len(data_plot))
print('dataPlot', data_plot.head(1))
plt.plot(data_plot['ds'][-500:], data_plot['y'][-500:])

## Kaggle energy

In [ ]:
raw_data = pd.read_csv('Load_history.csv')
raw_data = raw_data.head(len(raw_data)-10)
data = pd.wide_to_long(raw_data, stubnames='h', i=['zone_id', 'year', 'month','day'], j='hour').reset_index().rename(columns={'h':'y'})
data['ds'] = data['year'].astype(str) + '-' + data['month'].astype(str).str.zfill(2) + '-' + data['day'].astype(str).str.zfill(2) + ' ' + (data['hour']-1).astype(str).str.zfill(2) + ':00'
data['ds'] = pd.to_datetime(data['ds'])
data['y'] = data['y'].str.replace(',','').astype(float)
data = data[['zone_id', 'ds', 'y']]
data.columns = ['unique_id', 'ds', 'y']

In [ ]:
data.to_csv('new_data/energy_kaggle.csv', index=False)

In [ ]:
data_plot = data[data['unique_id']==data['unique_id'].unique()[np.random.randint(data['unique_id'].nunique())]]
print('len', len(data_plot))
print('dataPlot', data_plot.head(1))
plt.plot(data_plot['ds'][-500:], data_plot['y'][-500:])